# \> Множественное наследование (1/3)
[Хорошая статья про проблемы множественного наследования](https://www.geeksforgeeks.org/multiple-inheritance-in-python/)

Попробуем сделать множественное наследование аккуратно и так, как его обычно делают в популярных библиотеках.

Обычно множественное наследование используют в т.н. Mixins. Mixin (рус. миксины) - это класс, не имеющий полей и имеющий один или более не абстрактных методов (т.е. методов с реализацией). Обычно миксины используют для того, чтобы модульно раздавать различные функциональности различным объектам.

В лекции был пример с `Connectable` и `PostgresqlConnection`. `Connectable` как раз был одной из `Mixin`.

Вас перевели из тех.директора сложного проекта в бекенд-разработчика. Встала задача разработать класс, которому на вход будет приходить разобранный запрос от пользователя. Запрос имеет вид:
```python
# будет приходить запрос в виде словаря
request = {
  "cookies": {key_1: value_1, key_2: value_2, ...},
  "body": "a long time ago, in a Galaxy far, far away",
  "headers": {"content-type": "application/json", "Accept": "application/json"}
}

# и этот словарь будет передаваться в конструктор класса
handler = Handler(request)
```
Разным классам в приложении потребуется разная функциональность: кому-то потребуется проверять, есть ли в `headers` ключ `"Accept"`, кому-то потребуется читать `body`, а кому-то понадобится проверять пустоту `cookies`. Будут и классы, которым потребуется несколько возможностей сразу.

Напишите классы `ParsesCookies`, `ParsesBody`, `ParsesHeaders` по условиям:

1. Класс `ParsesCookies` имеет метод `cookies()`, возвращающий все по ключу `cookies` из словаря `self.request`.
2. Класс `ParsesCookies` имеет метод `is_authed()`, который будет проверять, что в словаре `cookies` будет ключ `auth_key` (ни в коем случае не используйте такую авторизацию в реальных проектах).
3. Класс `ParsesBody` имеет метод `body()`, возвращающий текст по ключу `body` в `self.request`.
4. Класс `ParsesHeaders` имеет метод `headers()`, возвращающий все по ключу `headers` из словаря `self.request`.
5. Класс `ParsesHeaders` имеет метод `need_json()`, который возвращает `True`, если в `headers` по ключу "content-type" лежит значение "application/json", иначе `False`.

In [17]:
class ParsesCookies:
    def cookies(self):
        return self.request.get('cookies')
    
    def is_authed(self):
        return 'auth_key' in self.request['cookies'].keys()

class ParsesBody:
    def body(self):
        return self.request.get('body')

class ParsesHeaders:
    def headers(self):
        return self.request.get('headers')
    
    def need_json(self):
        if self.request.get('headers').get('content-type') == 'application/json':
            return True
        else:
            return False

class Handler(ParsesCookies, ParsesBody, ParsesHeaders):
    def __init__(self, request):
        self.request = request

In [18]:
request = {
  "cookies": {'key_1': 'value_1', 'key_2': 'value_2'},
  "body": "a long time ago, in a Galaxy far, far away",
  "headers": {"content-type": "application/json", "Accept": "application/json"}
}
handler = Handler(request)

In [19]:
handler.cookies()

{'key_1': 'value_1', 'key_2': 'value_2'}

In [20]:
handler.is_authed()

False

In [21]:
handler.body()

'a long time ago, in a Galaxy far, far away'

In [22]:
handler.headers()

{'content-type': 'application/json', 'Accept': 'application/json'}

In [14]:
handler.need_json()

True